In [1]:
from pandas_datareader import data
import datetime
from bokeh.plotting import figure, show, output_file
from bokeh.embed import components
from bokeh.resources import CDN

In [2]:
start=datetime.datetime(2020,6,1)
end=datetime.datetime(2021,6,18)
df=data.DataReader(name="AAPL", data_source="yahoo", start=start, end=end)

In [3]:
#method 1
#date_gain=df.index[df.Close>df.Open]
#date_loss=df.index[df.Close<df.Open]

In [4]:
#method 2 add new data column
def inc_dec(c,o):
    if c>o:
        value = "Increase"
    elif c<o:
        value = "Decrease"
    else:
        value = "Equal"
    return value

df["Status"]=[inc_dec(c,o) for c,o in zip(df.Close, df.Open)]

df["Middle"]=(df.Close+df.Open)/2
df["Height"]=abs(df.Open-df.Close)

In [5]:
p=figure(x_axis_type='datetime', width=1000,height=300, sizing_mode="scale_width")
p.title = "Candlestick Chart"
p.grid.grid_line_alpha=0.3

hours_12=12*60*60*1000

#method 1
#p.rect(date_gain, (df.Close+df.Open)/2, hours_12, abs(df.Open-df.Close), 
#       fill_color="green", line_color="black")
#p.rect(date_loss, (df.Close+df.Open)/2, hours_12, abs(df.Open-df.Close), 
#       fill_color="red", line_color="black")

#method 2
#the order of plotting matters. it will build on top layer of previous
p.segment(df.index, df.High, df.index, df.Low)
p.rect(df.index[df.Status=="Increase"], df.Middle[df.Status=="Increase"], hours_12, df.Height[df.Status=="Increase"], 
       fill_color="green", line_color="black")
p.rect(df.index[df.Status=="Decrease"], df.Middle[df.Status=="Decrease"], hours_12, df.Height[df.Status=="Decrease"], 
       fill_color="red", line_color="black")

#local output
output_file("stock.html")
show(p)

#live web output
#get html script from p
script1, div1 = components(p)
cdn_js = CDN.js_files
cdn_css=CDN.css_files